https://www.section.io/engineering-education/classification-model-using-bert-and-tensorflow/

In [1]:
import tensorflow_hub as hub
import pandas as pd
import numpy as np
import tensorflow_text as text
from tensorflow.keras import layers, Model, metrics
from tensorflow import string
from transformers import BertTokenizer
from sklearn.preprocessing import LabelEncoder 
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('Data/phraseBank_dataset.csv', encoding="ISO-8859-1",header=None)
df.columns = ['class','sentence']
df.head(5)

,class,sentence
0,neutral,"According to Gran , the company has no plans t..."
1,neutral,Technopolis plans to develop in stages an area...
2,negative,The international electronic industry company ...
3,positive,With the new production plant the company woul...
4,positive,According to the company 's updated strategy f...


In [3]:
label_encoder = LabelEncoder()
df['labels'] = label_encoder.fit_transform(df['class'])
df.head(5)

,class,sentence,labels
0,neutral,"According to Gran , the company has no plans t...",1
1,neutral,Technopolis plans to develop in stages an area...,1
2,negative,The international electronic industry company ...,0
3,positive,With the new production plant the company woul...,2
4,positive,According to the company 's updated strategy f...,2


In [4]:
X= df[['sentence']]
y = df['labels']

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)
y_train

432     1
3071    1
289     2
2588    1
4121    1
       ..
4426    0
466     2
3092    1
3772    1
860     2
Name: labels, Length: 3876, dtype: int32

In [6]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [7]:
text_input = layers.Input(shape=(), dtype=string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

In [8]:
l = layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = layers.Dense(1, activation='sigmoid', name="output")(l)

In [9]:
model = Model(inputs=[text_input], outputs = [l])


In [10]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_type_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128),                                                          
                                 'input_word_ids':                                                
                                (None, 128)}                                                  

In [11]:
METRICS = [
      metrics.BinaryAccuracy(name='accuracy'),
      metrics.Precision(name='precision'),
      metrics.Recall(name='recall')
]

model.compile(optimizer='adam',
 loss='binary_crossentropy',
 metrics=METRICS)

In [12]:
model.fit(X_train, y_train, epochs=1)

122/122 [==============================] - 1231s 10s/step - loss: -2.3226 - accuracy: 0.5869 - precision: 0.8722 - recall: 0.9829


In [15]:
model.evaluate(X_test,y_test)

31/31 [==============================] - 304s 10s/step - loss: -5.6944 - accuracy: 0.5876 - precision: 0.8865 - recall: 0.9988


[-5.694360256195068,
 0.5876288414001465,
 0.8864809274673462,
 0.9988372325897217]

In [16]:
model.save('bert-sentiment-showcase-from-phraseBank.h5', overwrite=True, include_optimizer=True)